# User Item Biases
* Computes the UserItemBias alpha for the recommendee
* See `../TrainingAlphas/UserItemBiases.ipynb` for algorithm details

In [1]:
source = "UserItemBiases";

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

## Alternating Least Squares

In [3]:
function get_residuals!(users, items, ratings, u, a, ρ, Ω)
    for row = 1:length(users)
        i = users[row]
        j = items[row]
        r = ratings[row]
        ρ[i] += r - a[j]
        Ω[i] += 1
    end
    ρ, Ω
end

# todo move to utils
function thread_range(n)
    tid = Threads.threadid()
    nt = Threads.nthreads()
    d, r = divrem(n, nt)
    from = (tid - 1) * d + min(r, tid - 1) + 1
    to = from + d - 1 + (tid ≤ r ? 1 : 0)
    from:to
end

function update_users!(users, items, ratings, u, a, λ_u, ρ, Ω)
    Threads.@threads for t = 1:Threads.nthreads()
        range = thread_range(length(ratings))
        ρ[:, Threads.threadid()] .= 0
        Ω[:, Threads.threadid()] .= 0
        @views get_residuals!(
            users[range],
            items[range],
            ratings[range],
            u,
            a,
            ρ[:, Threads.threadid()],
            Ω[:, Threads.threadid()],
        )
    end
    ρ = sum(ρ, dims = 2)
    Ω = sum(Ω, dims = 2)

    μ = mean(u)
    Threads.@threads for i = 1:length(u)
        u[i] = (ρ[i] + μ * λ_u) / (Ω[i] + λ_u)
    end
end;

In [4]:
function train_model(training, λ_u, a, stop_criteria)
    @debug "training model with parameters [$λ_u]"
    users, items, ratings = training.user, training.item, training.rating
    u = zeros(eltype(λ_u), maximum(users))

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())

    while !stop!(stop_criteria, [u])
        update_users!(users, items, ratings, u, a, λ_u, ρ_u, Ω_u)
        @debug u
    end
    u
end;

In [5]:
function make_prediction(users, items, u, a)
    r = zeros(eltype(u), length(users))
    u_mean = mean(u)
    a_mean = mean(a)
    for i = 1:length(r)
        if users[i] > length(u)
            r[i] += mean(u)
        else
            r[i] += u[users[i]]
        end
        if items[i] > length(a)
            r[i] += mean(a)
        else
            r[i] += a[items[i]]
        end
    end
    r
end;

## Recommendee predictions

In [6]:
function compute_alpha()
    training = get_split("recommendee")
    params = read_params(source)

    training.user .= 1 # relabel ids so that recommendee -> 1
    stop_criteria = convergence_stopper(1e-9)
    u = train_model(training, params["λ"][1], params["a"], stop_criteria)

    model(items) = make_prediction(fill(1, length(items)), items, u, params["a"])
    write_recommendee_alpha(model)
end;

In [7]:
compute_alpha();

[ Debug: 20220129 21:33:04 training model with parameters [2.2125266]
[ Debug: 20220129 21:33:05 Float32[6.2695413]
[ Debug: 20220129 21:33:06 Float32[6.300216]
[ Debug: 20220129 21:33:06 Float32[6.3003664]
[ Debug: 20220129 21:33:06 Float32[6.300367]
[ Debug: 20220129 21:33:06 Float32[6.300367]
